In [1]:
"""
Technical Indicators for Stock Market 
"""

'\nTechnical Indicators for Stock Market \n'

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import date

import yfinance as yf

In [6]:
START = '2018-01-01'
TODAY = date.today().strftime("%Y-%m-%d")

selected_stocks = "AAPL"

def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

data = load_data(selected_stocks)
data.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,41.248272,102223600
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,41.241089,118071600
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,41.432659,89738400
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.904385,94640000
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.748737,82271200
